# Group 4: Random Forest Model and Evaluation

In [33]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image


## Loading and Preprocessing Loans Encoded Data

In [34]:
# Loading data
file_path = Path("..\Group4\sentiment_closing.csv")
df_sentiment_closing = pd.read_csv('sentiment_closing.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)
df_sentiment_closing.head()




,title_compound,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2022-02-04,0.000000,0.137523,0.147592,0.137250,0.147503,0.147503,580740990
2022-02-05,0.109560,0.147533,0.150140,0.145410,0.147141,0.147141,587009429
2022-02-06,0.109560,0.147140,0.154775,0.145258,0.153770,0.153770,757004142
2022-02-07,0.074822,0.153803,0.171586,0.151741,0.165578,0.165578,1791358822
2022-02-08,0.193133,0.165606,0.169145,0.153709,0.158405,0.158405,1190116000


In [35]:
# Define features set
X = df_sentiment_closing.copy()
X.drop("title_compound", axis=1, inplace=True)
X.head(50)



,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-04,0.137523,0.147592,0.137250,0.147503,0.147503,580740990
2022-02-05,0.147533,0.150140,0.145410,0.147141,0.147141,587009429
2022-02-06,0.147140,0.154775,0.145258,0.153770,0.153770,757004142
2022-02-07,0.153803,0.171586,0.151741,0.165578,0.165578,1791358822
2022-02-08,0.165606,0.169145,0.153709,0.158405,0.158405,1190116000
2022-02-09,0.158357,0.160704,0.155035,0.159123,0.159123,778670798
2022-02-10,0.159145,0.160437,0.151497,0.151889,0.151889,1053630584
2022-02-11,0.151895,0.154069,0.143349,0.144847,0.144847,776730555
2022-02-12,0.144856,0.147013,0.141678,0.144405,0.144405,602699408


In [36]:
# Define target vector
# y = df_sentiment_closing["title_compound"].ravel()
# y = df_sentiment_closing["title_compound"]
y = df_sentiment_closing["title_compound"].values.reshape(-1,1)
y[:5]



array([[0.        ],
       [0.10956   ],
       [0.10956   ],
       [0.07482222],
       [0.19313333]])

In [37]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)



In [38]:
# Creating StandardScaler instance
scaler = StandardScaler()



In [39]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)



In [40]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [41]:
# Confirm scaled values...for fun
df_X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
round(df_X_train_scaled.mean(), 0)


Open        -0.0
High        -0.0
Low         -0.0
Close        0.0
Adj Close    0.0
Volume      -0.0
dtype: float64

In [42]:
round(df_X_train_scaled.std(), 0)


Open         1.0
High         1.0
Low          1.0
Close        1.0
Adj Close    1.0
Volume       1.0
dtype: float64

## Fitting the Random Forest Model

In [43]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)



Once the random forest model is created, it is fitted with the training data.

In [45]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


C:\Users\Queenbee\AppData\Local\Temp/ipykernel_15648/3885632750.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model = rf_model.fit(X_train_scaled, y_train)


ValueError: Unknown label type: 'continuous'

## Making Predictions Using the Random Forest Model

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)


## Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)



In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_



In [ ]:
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)



In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

